<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/tf/t_word_embeddings_textvectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word embeddings

Word embeddings give us a way to use an efficient, dense representation in which similar words have a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a dense vector of floating point values (the length of the vector is a parameter you specify). Instead of specifying the values for the embedding manually, they are trainable parameters (weights learned by the model during training, in the same way a model learns weights for a dense layer). It is common to see word embeddings that are 8-dimensional (for small datasets), up to 1024-dimensions when working with large datasets. A higher dimensional embedding can capture fine-grained relationships between words, but takes more data to learn.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding2.png?raw=1" alt="Diagram of an embedding" width="400"/>

Above is a diagram for a word embedding. Each word is represented as a 4-dimensional vector of floating point values. Another way to think of an embedding is as "lookup table". After these weights have been learned, you can encode each word by looking up the dense vector it corresponds to in the table.

## Setup

In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from datetime import datetime
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

tf.__version__

'2.3.0'

### Download the IMDb Dataset
You will use the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) through the tutorial. 

In [2]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file(
    'aclImdb_v1.tar.gz',
    url,
    untar=True,
    cache_dir='.',
    cache_subdir=''
)

dir_name = os.path.dirname(dataset)
dataset_dir = os.path.join(dir_name, 'aclImdb')
print(dataset_dir)
os.listdir(dataset_dir)

./aclImdb


['test', 'train', 'imdb.vocab', 'README', 'imdbEr.txt']

Take a look at the `train/` directory. It has `pos` and `neg` folders with movie reviews labelled as positive and negative respectively. You will use reviews from `pos` and `neg` folders to train a binary classification model.

In [3]:
train_dir = os.path.join(dataset_dir, 'train')
train_dir

'./aclImdb/train'

In [4]:
remove_dir = os.path.join(train_dir, 'unsup')
try:
  shutil.rmtree(remove_dir)
except:
  print('unsup folder was not found.')

Next, create a `tf.data.Dataset` using `tf.keras.preprocessing.text_dataset_from_directory`. 

Use the `train` directory to create both train and validation datasets with a split of 20% for validation.

In [5]:
batch_size = 1024
seed = 123
train_ds = text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed 
)
val_ds = text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed 
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


Take a look at a few movie reviews and their labels `(1:possitive, 0:negative)` from the train dataset.

In [6]:
text_batch, label_batch = next(iter(train_ds.take(1)))
[(label_batch[i].numpy(), text_batch[i].numpy()) for i in range(2)]

[(0,
  b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"),
 (1,
  b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This movie is a tad bit "grease"

### Configure the dataset for performance

These are two important methods you should use when loading data to make sure that I/O does not become blocking.

`.cache()` keeps data in memory after it's loaded off disk. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache, which is more efficient to read than many small files.

`.prefetch()` overlaps data preprocessing and model execution while training. 

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def optimize_ds(ds):
  ds.cache().prefetch(buffer_size=AUTOTUNE)
  return ds

In [11]:
train_ds = optimize_ds(train_ds)
val_ds = optimize_ds(val_ds)

## Using the Embedding layer

Keras makes it easy to use word embeddings. Take a look at the [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer.

The Embedding layer can be understood as a lookup table that maps from integer indices (which stand for specific words) to dense vectors (their embeddings). The dimensionality (or width) of the embedding is a parameter you can experiment with to see what works well for your problem, much in the same way you would experiment with the number of neurons in a Dense layer.


In [12]:
embedding_layer = tf.keras.layers.Embedding(1000, 5)

When you create an Embedding layer, the weights for the embedding are randomly initialized (just like any other layer). During training, they are gradually adjusted via backpropagation. Once trained, the learned word embeddings will roughly encode similarities between words (as they were learned for the specific problem your model is trained on).

If you pass an integer to an embedding layer, the result replaces each integer with the vector from the embedding table:

In [13]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[-1.3687693e-02,  2.0714615e-02,  2.4951920e-03, -2.9708743e-03,
         8.1034750e-04],
       [ 4.5248915e-02, -4.7238577e-02, -1.4678389e-04, -2.5926996e-02,
        -9.1179609e-03],
       [-1.6784668e-02,  5.4322183e-05, -3.0800510e-02, -6.8064108e-03,
         2.4095774e-03]], dtype=float32)